## Simulating the game with options of playing strategically or randomly.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive

def game_simulation(n=4, sims=100, random=0, first=0):
    p0_wins = 0  

    for i in range(sims):
        matrixarray = np.full(n * n, 2)  

        if random == 0:
            half = np.random.choice(n * n, (n * n) // 2, replace=False)
            if first == 0:
                matrixarray[half] = 1
                matrixarray[matrixarray == 2] = 0
            else:
                matrixarray[half] = 0
                matrixarray[matrixarray == 2] = 1
        else:
            ncounter = 0

            if first == 0:
                if n % 2 == 0:
                    p0index = np.random.choice(np.where(matrixarray == 2)[0])
                else:
                    p0index = (n-1)//2
                matrixarray[p0index] = 0
                ncounter += 1

            while ncounter < n * n:
                p1index = np.random.choice(np.where(matrixarray == 2)[0])
                matrixarray[p1index] = 1
                ncounter += 1

                if ncounter >= n * n:
                    break

                if n % 2 == 0:
                    p0index = p1index - 2 * (p1index % n) + (n - 1)
                else:
                    mid = (n - 1) // 2
                    if (p1index % n) == mid:
                        for r in range(n):
                            p0index = r * n + mid
                            if matrixarray[p0index] == 2:
                                break
                    else:
                        p0index = p1index - 2 * (p1index % n) + (n - 1)

                if not (0 <= p0index < n * n and matrixarray[p0index] == 2):
                    available_moves = np.where(matrixarray == 2)[0]
                    valid_moves = [move for move in available_moves 
                                   if 0 <= (move - 2 * (move % n) + (n - 1)) < n * n 
                                   and matrixarray[move - 2 * (move % n) + (n - 1)] == 2]
                    if valid_moves:
                        p0index = np.random.choice(valid_moves)
                
                matrixarray[p0index] = 0
                ncounter += 1

        A = np.reshape(matrixarray, (n, n))
        det = np.linalg.det(A)

        if np.isclose(det, 0, atol=1e-8):
            p0_wins += 1

    win_ratio = p0_wins / sims
    plt.figure(figsize=(5, 3))
    plt.bar(["Player 0 Wins", "Player 1 Wins"], [win_ratio, 1 - win_ratio], color=["blue", "red"])
    plt.ylim(0, 1)
    plt.ylabel("Proportion of Wins")
    plt.title(f"n={n}, sims={sims}, Strategy={random}, First Player={first}")
    plt.show()

    print("Sample matrix for n =", n)
    print(A)
    print("Determinant:", np.round(det))
    _,zs = np.shape(np.where(matrixarray == 0))
    print("Number of zeros:",zs)

interactive_plot = interactive(game_simulation, 
    n=widgets.IntSlider(min=4, max=30, step=1, value=4, description="Grid Size (n)"),
    sims=widgets.IntSlider(min=10, max=1000, step=10, value=10, description="Simulations"),
    random=widgets.ToggleButtons(options=[0, 1], value=1, description="Player 0 Strategy Off/On?"),
    first=widgets.ToggleButtons(options=[0, 1], value=0, description="First Player"))

interactive_plot

interactive(children=(IntSlider(value=4, description='Grid Size (n)', max=30, min=4), IntSlider(value=10, desc…